In [54]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [55]:
def euclidean_distance(tensor_a, tensor_b, dim = 0):
    return (tensor_b - tensor_a).pow(2).sum(dim).sqrt()

In [56]:
input = torch.randn(256, 128, 1, requires_grad=True)
target = torch.rand(256, 1, 1)
target = target.mul(9)
target = target.absolute()
target = target.round()

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [57]:
input[0]

tensor([[ 6.8408e-01],
        [ 1.3120e+00],
        [ 1.7134e-01],
        [ 2.0987e-01],
        [-8.3670e-01],
        [-8.3740e-01],
        [-9.5316e-01],
        [ 1.1270e+00],
        [ 1.3052e+00],
        [ 6.9400e-01],
        [ 1.0976e+00],
        [-5.9098e-01],
        [-2.7446e-02],
        [-6.3795e-01],
        [-6.7423e-01],
        [-1.2494e+00],
        [ 6.8715e-01],
        [-5.1271e-01],
        [-7.6053e-02],
        [-8.0101e-01],
        [-9.5664e-01],
        [-8.5392e-01],
        [ 6.1499e-02],
        [-1.0228e+00],
        [ 1.0078e+00],
        [-3.4737e+00],
        [-7.3897e-01],
        [ 4.5608e-02],
        [ 8.7347e-01],
        [ 1.1483e+00],
        [ 3.2783e-02],
        [-1.5879e-01],
        [ 5.7602e-01],
        [ 1.9517e+00],
        [-7.8750e-01],
        [-3.4020e-01],
        [ 7.7135e-01],
        [ 3.3376e-01],
        [-1.5931e-01],
        [ 8.2835e-01],
        [-4.3961e-01],
        [ 1.6420e+00],
        [ 1.3105e+00],
        [ 6

In [58]:
def prepare(input, target, classes):
    batch_size = input.shape[0]
    # batch_size
    centroids = []
    for _class in classes:
        class_tensors = []
        for i in range(batch_size):
            if target[i] == _class:
                class_tensors.append(input[i])
        stacked_tensor = torch.stack(class_tensors)
        centroids.append(torch.mean(stacked_tensor, dim = 0))

    # self.centroids = torch.stack(centroids)

    # Jedes Element in centroids ist weiterhin ableitbar, aber centroids darf selbst nicht weiterverwendet werden.

    distances = []
    for centroid_a in centroids:
        local_distances = []
        for centroid_b in centroids:
            if not torch.equal(centroid_a, centroid_b):
                local_distances.append(euclidean_distance(centroid_a, centroid_b))
        local_tensor = torch.stack(local_distances)
        distances.append(torch.mean(local_tensor))      

    # (centroids[0] - centroids[1]).pow(2).sum(0).sqrt()
    return distances

In [59]:
dists = prepare(input, target, classes)

In [76]:
# # sample_losses = torch.Tensor([0])
# for sample, label in zip(input, target):
#     # sample_losses.append(dists[int(label)])
#     try:
#         sample_losses = torch.cat((sample_losses, dists[int(label)]), 0)
#     except:
#         sample_losses = dists[int(label)]
# # loss_tensor = torch.stack(sample_losses)
# sample_losses

torch.stack(dists)[0]

tensor(3.3633, grad_fn=<SelectBackward0>)